In [40]:
import pandas as pd
import zipfile
import os
import time
from tqdm import tqdm
import shutil
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [41]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [42]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    print(os.walk(path))
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))

    return files_add
path="pcaps"
files_add=find_the_way(path,'.pcap')
files_add

<generator object walk at 0x000002C5BA28F120>


['pcaps\\dos-synflooding-5-dec.pcap']

In [ ]:
def flag_fixer(file):

    IP_flags={'0': 1,
 '<Flag 0 ()>': 2,
 '<Flag 2 (DF)>': 3,
 '<Flag 1 (MF)>': 4,
 '<Flag 3 (MF+DF)>': 40,
 '<Flag 4 (evil)>': 41,
 '<Flag 5 (MF+evil)>': 42,
 '<Flag 6 (DF+evil)>': 43,
 '<Flag 7 (MF+DF+evil)>': 44}
    df=pd.read_csv(file)
    IP_flags
    
    df["IP_flags"]=df["IP_flags"].map(IP_flags.get)

    df.to_csv(file,index=None)





# PCAP2CSV

In [44]:
from scapy.all import*

In [45]:
import math
import pandas as pd
import os
import numpy as np

In [46]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [47]:
def shannon(data):
    LOG_BASE = 2
   # We determine the frequency of each byte
   # in the dataset and if this frequency is not null we use it for the
   # entropy calculation
    dataSize = len(data)
    ent = 0.0
    freq={}
    for c in data:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1
   # to determine if each possible value of a byte is in the list
    for key in freq.keys():
        f = float(freq[key])/dataSize
        if f > 0: # to avoid an error for log(0)
            ent = ent + f * math.log(f, LOG_BASE)
    return -ent

def pre_entropy(payload):

    characters=[]
    for i in payload:
            characters.append(i)
    return shannon(characters)


In [48]:
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]# private port list (0-Reserved,53-DNS, 67-BOOTP server, 68-BOOTP client...)
    if port in port_list: #Is the port number in the list?
        return port_list.index(port)+1 # return the port's index number in the list (actually with index+1)
    elif 0 <= port <= 1023: # return 11 if the port number is in the range 0 :1023
        return 11
    elif  1024 <= port <= 49151 : # return 12 if the port number is in the range 1024:49151
        return 12
    elif 49152 <=port <= 65535 :# return 13 if the port number is in the range 49152:65535
        return 13
    else:# return 0 if no previous conditions are met
        return 0


def port_1023(port):
    if 0 <= port <= 1023:
        return port
    elif  1024 <= port <= 49151 :
        return 2
    elif 49152 <=port <= 65535 :
        return 3
    else:
        return 0

In [49]:
header="ts,IP_src,IP_dst,pck_size,IP_flags,IP_Z,IP_MF,IP_DF,TCP_dataofs,TCP_FIN,TCP_SYN,TCP_RST,TCP_PSH,TCP_ACK,TCP_URG,TCP_ECE,TCP_CWR,TCP_window,sport_class,sport_bare,dport_bare,TCP_sport,TCP_dport,UDP_sport,UDP_dport\n"

In [50]:
#flags
#TCP
FIN = 0x01
SYN = 0x02
RST = 0x04
PSH = 0x08
ACK = 0x10
URG = 0x20
ECE = 0x40
CWR = 0x80
#IP
Z = 0x00
MF= 0x01
DF= 0x02


# Individual paket pased faetures

In [51]:
def pcap_parser(oldname):
    piece_size=50000
    new_name=oldname.replace(".pcap","_smaller.pcap")
    command='C:/\"Program Files\"/Wireshark/editcap.exe -c '+str(piece_size)+" \""+oldname+"\" "+new_name
    os.system(command)
    parsed=find_the_way("./",new_name[6:-5])
    return parsed,new_name[6:-5]


In [52]:
def merged_csv(name,keyword):
    for merger in ["_FE.csv","_WS.csv"]:
        merged_name=f"{name[:-4]}{merger}"
        csv_files=find_the_way("./",keyword,merger)
        df=pd.read_csv(csv_files[0])
        col_names=list(df.columns)

        empty = pd.DataFrame(columns=col_names)
        empty.to_csv(merged_name, mode="w", index=False)#,header=False)


        for j in csv_files:

            df=pd.read_csv(j)
            #print("name and shape of dataframe :",i,df.shape)
            df.to_csv(merged_name, mode="a", index=False,header=False)
            os.remove(j)
            try:
                os.remove(j.replace("_FE.csv",".pcap"))
            except:pass

            #print("\n\n\nname and shape of dataframe :",name,df.shape)

In [ ]:
outputfolder="FE"
folder(outputfolder)
for sayac,i in enumerate (files_add):
    print(f"{sayac+1}/{len(files_add)}-{i}")
    file_list=[i]
    for isim in file_list:

        filename=isim.replace(".pcap","_FE.csv")
        ths = open(filename, "w")
        ths.write(header)
        print(header)
        pkt = rdpcap(isim)
        for j in tqdm(pkt):
            ts=float(j.time) #
            try:pck_size=j.len #
            except:pck_size=0
           
            if j.haslayer(IP): #
                IP_Z = 0
                IP_MF= 0
                IP_DF= 0 #
             
                IP_flags=j[IP].flags#

               

                if IP_flags & Z:IP_Z = 1
                if IP_flags & MF:IP_MF = 1
                if IP_flags & DF:IP_DF = 1 #
                
                IP_src=j[IP].src#IP_adresses.index(j[IP].src)+1
                IP_dst=j[IP].dst#IP_adresses.index(j[IP].dst)+1



            else:
                IP_Z = 0
                IP_MF= 0
                IP_DF= 0

                # IP_version=0
                # IP_ihl=0
                # IP_tos=0
                # IP_len=0
                # IP_id=0
                IP_flags=0
                # IP_frag=0
                # IP_ttl=0
                # IP_proto=0
                # IP_chksum=0
                IP_src=0
                IP_dst=0
                # IP_options=0
                # IP_add_count=0


            if j.haslayer(TCP):
                TCP_FIN = 0
                TCP_SYN = 0 #
                TCP_RST = 0
                TCP_PSH = 0
                TCP_ACK = 0 #
                TCP_URG = 0
                TCP_ECE = 0
                TCP_CWR = 0
                TCP_sport=j[TCP].sport
                TCP_dport=j[TCP].dport
                # TCP_seq=j[TCP].seq
                # TCP_ack=j[TCP].ack
                TCP_dataofs=j[TCP].dataofs #
                # TCP_reserved=j[TCP].reserved
                TCP_flags=j[TCP].flags

                TCP_window=j[TCP].window #
                # TCP_chksum=j[TCP].chksum
                # TCP_urgptr=j[TCP].urgptr
                # TCP_options=j[TCP].options
                # TCP_options= str(TCP_options).replace(",","-")
                # if TCP_options!="0":
                #     TCP_options=1
                # else:
                #     TCP_options=0




                #if TCP_flags not in tcpf:
                    #tcpf.append(TCP_flags)
                #print(TCP_options)
                if TCP_flags & FIN:TCP_FIN = 1
                if TCP_flags & SYN:TCP_SYN = 1
                if TCP_flags & RST:TCP_RST = 1
                if TCP_flags & PSH:TCP_PSH = 1
                if TCP_flags & ACK:TCP_ACK = 1
                if TCP_flags & URG:TCP_URG = 1
                if TCP_flags & ECE:TCP_ECE = 1
                if TCP_flags & CWR:TCP_CWR = 1




            else:
                TCP_sport=0
                TCP_dport=0
                # TCP_seq=0
                # TCP_ack=0
                TCP_dataofs=0
                # TCP_reserved=0
                TCP_flags=0
                TCP_window=0
                # TCP_chksum=0
                # TCP_urgptr=0
                # TCP_options=0
                # TCP_options=0
                TCP_FIN = 0
                TCP_SYN = 0
                TCP_RST = 0
                TCP_PSH = 0
                TCP_ACK = 0
                TCP_URG = 0
                TCP_ECE = 0
                TCP_CWR = 0

            if j.haslayer(UDP):
                UDP_sport=j[UDP].sport
                UDP_dport=j[UDP].dport
                UDP_len=j[UDP].len
                UDP_chksum=j[UDP].chksum
            else:
                UDP_sport=0
                UDP_dport=0
                UDP_len=0
                UDP_chksum=0
           
            sport_class=port_class(TCP_sport+UDP_sport) #
            
            sport_bare=TCP_sport+UDP_sport
            dport_bare=TCP_dport+UDP_dport#port_class(TCP_dport+UDP_dport)


            try:Mac=j.src ############# thiếu trong xử lý py nếu lỗi thì thêm vào
            except:Mac= j.addr1
            line=[ts,  IP_src,
            IP_dst, pck_size,
            IP_flags,IP_Z, IP_MF,IP_DF  ,#
            TCP_dataofs,TCP_FIN,
            TCP_SYN, TCP_RST, TCP_PSH, TCP_ACK,  TCP_URG, TCP_ECE, TCP_CWR   , TCP_window,
            sport_class,sport_bare, dport_bare,TCP_sport,TCP_dport,
            UDP_sport,
            UDP_dport
            ]
            #print(line)
            line=str(line).replace("[","")
            line=str(line).replace("]","")
            #line=str(line).replace("\',","-")
            line=str(line).replace(", ",",")
            line=str(line).replace("\'","")
            line=str(line).replace("None","0")
            # if label:
            #     Label=df[label_count]
            # else:
            #     Label="No_Label"
            ths.write(str(line)+f"\n")
            # label_count+=1
            
        name=isim.replace("\\","/")
        output=filename.replace("_FE.csv","_WS.csv")

        if " " not in name:
            command=f"tshark -r {name}  -T fields -e _ws.col.Source -e _ws.col.Destination  
            -e _ws.col.Protocol    -E header=y -E separator=, -E quote=d -E occurrence=f >{output}"
        else:
            command=f"tshark -r \"{name}\" -T fields -e _ws.col.Source -e _ws.col.Destination  
            -e _ws.col.Protocol    -E header=y -E separator=, -E quote=d -E occurrence=f >\"{output}\""
        os.system(command)
        ths.close()
    if len(file_list)>1:
        merged_csv(i,keyword)



1/1-pcaps\dos-synflooding-5-dec.pcap
ts,IP_src,IP_dst,pck_size,IP_flags,IP_Z,IP_MF,IP_DF,TCP_dataofs,TCP_FIN,TCP_SYN,TCP_RST,TCP_PSH,TCP_ACK,TCP_URG,TCP_ECE,TCP_CWR,TCP_window,sport_class,sport_bare,dport_bare,TCP_sport,TCP_dport,UDP_sport,UDP_dport



100%|██████████| 11366/11366 [00:00<00:00, 14874.27it/s]


# WIRESHARK FEATURES dst,src and Protocol added

In [54]:
name_list=find_the_way('./pcaps','_FE.csv')
name_list

<generator object walk at 0x000002C5B7E9E5A0>


['./pcaps\\dos-synflooding-5-dec_FE.csv']

In [ ]:
name_list=find_the_way('./pcaps','_FE.csv')
for i in tqdm(name_list):
    print(i)
    df=pd.read_csv(i)
    WS_s_d=i.replace("_FE.csv","_WS.csv")
    s_d=pd.read_csv(WS_s_d)
    df["WS_src"]=s_d["_ws.col.Source"]
    df["WS_dst"]=s_d["_ws.col.Destination"]
    df["Protocol"]=s_d["_ws.col.Protocol"]
    # y=df["Label"]
    # del df["Label"]
    # df["Label"]=y
    df.to_csv(i,index=None)
    flag_fixer(i)
    os.remove(WS_s_d)

<generator object walk at 0x000002C5B7E9E5A0>


  0%|          | 0/1 [00:00<?, ?it/s]

./pcaps\dos-synflooding-5-dec_FE.csv
./pcaps\dos-synflooding-5-dec_FE.csv


100%|██████████| 1/1 [00:00<00:00,  4.84it/s]


In [56]:
name_list=find_the_way('./pcaps','_FE.csv')

<generator object walk at 0x000002C5B7E9E5A0>


# NEW FEATURES: dst_IP_diversity, dst_port_diversity, src_IP_diversity IP_add_count

In [ ]:
for sayac,i in enumerate (name_list):
    print(f"{sayac+1}/{len(name_list)}-{i}")
    dst_IP_diversity=[]
    dst_port_diversity=[]
    src_IP_diversity=[]
    src_IP_v_dst_IP={}
    src_IP_v_dst_port={}
    df=pd.read_csv(i)
    count=0
    for j in tqdm(range(len(df))):
        if df['IP_src'][j] in src_IP_v_dst_IP:
            if df['IP_dst'][j] not in src_IP_v_dst_IP[df['IP_src'][j]]:
                src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        else:
            src_IP_v_dst_IP[df['IP_src'][j]]=[]
            src_IP_v_dst_IP[df['IP_src'][j]].append(df['IP_dst'][j])
        dst_IP_diversity.append(len(src_IP_v_dst_IP[df['IP_src'][j]]))
        ###############################################################################
        #  gắn port tương tự như IP
        if df['IP_src'][j] in src_IP_v_dst_port:
            if df['dport_bare'][j] not in src_IP_v_dst_port[df['IP_src'][j]]:
                src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        else:
            src_IP_v_dst_port[df['IP_src'][j]]=[]
            src_IP_v_dst_port[df['IP_src'][j]].append(df['dport_bare'][j])
        dst_port_diversity.append(len(src_IP_v_dst_port[df['IP_src'][j]]))
   
    df["dst_IP_diversity"]=dst_IP_diversity
    df["dst_port_diversity"]=dst_port_diversity
    y=df["Label"]
    del df["Label"]
    df["Label"]=y
    df.to_csv(i,index=None)


1/1-./pcaps\dos-synflooding-5-dec_FE.csv


100%|██████████| 11366/11366 [00:00<00:00, 16682.20it/s]


# Sliding Windows Features

In [65]:
outputfolder="FE"
folder(outputfolder)

In [66]:
files_add=find_the_way('./pcaps','FE.csv')
for i in files_add:
    os.rename(i,i.replace("./pcaps",'./FE'))

<generator object walk at 0x000002C5B7E9F120>


In [67]:
# Tính hiệu số giữa các giá trị liên tiếp trong danh sách x.
def diff(x):
    x = pd.Series(x)
    result=x.diff()
    result=result.fillna(0)
    return result
# Tính giá trị trung bình di động (rolling mean) với kích thước cửa sổ WS
# Trả về danh sách trung bình di động.
def mean(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).mean()
    result=result.fillna(result[WS-1])
    return result
# Tính độ lệch chuẩn di động (rolling standard deviation) với kích thước cửa sổ WS
# Tương tự hàm mean(x, WS), nhưng tính độ lệch chuẩn thay vì trung bình.
def std(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).std()
    result=result.fillna(result[WS-1])
    return result
# Tính trung bình mở rộng (expanding mean) cho tất cả giá trị từ đầu đến vị trí hiện tại.
# Trả về danh sách trung bình mở rộng.
def mean_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).mean()
    return result
# Tính độ lệch chuẩn mở rộng (expanding standard deviation).
def std_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).std() # Tính độ lệch chuẩn cho tất cả giá trị từ đầu đến vị trí hiện tại.
    result=result.fillna(0)
    return result
# Tính tổng mở rộng (expanding sum).
# Tương tự mean_EW(x), nhưng tính tổng thay vì trung bình.
def sum_of_EW(x):
    x = pd.Series(x)
    result=x.expanding(min_periods=1).sum()
    return result
# Tính phương sai di động (rolling variance) với kích thước cửa sổ WS.
def var(x,WS):
    x = pd.Series(x)
    result=x.rolling(WS).var()
    result=result.fillna(result[WS-1])
    return result
# Tính tổng tích lũy cho các giá trị 1 trong danh sách x.
# Trả về danh sách với tổng tích lũy của các giá trị 1.
def flagsum(x):
    total_f=0
    result=[]
    for i in x:
        if i==1:
            total_f+=1
        result.append(total_f)
    return result
# Tính số lượng cổng (port) duy nhất đã xuất hiện trong danh sách x tính đến từng thời điểm.
# Trả về danh sách số lượng cổng duy nhất.
def portsum(x):
    total_ports=[]
    result=[]
    for i in x:
        if i not in total_ports:
            total_ports.append(i)
        result.append(len(total_ports))
    return result     

In [68]:
files_add=find_the_way('./FE','FE.csv')
files_add

<generator object walk at 0x000002C5B7E9F120>


['./FE\\dos-synflooding-5-dec_FE.csv']

In [ ]:
outputfolder="SW"
folder(outputfolder)
for windows_size in [2,6,9]:
    for j in files_add:
        print(j[5:])
        df=pd.read_csv(j)#,usecols=n)
        df.drop(columns=["UDP_sport", "TCP_sport", "TCP_dport"], inplace=True)
        df.to_csv("temp.csv")
        df=pd.read_csv("temp.csv")
        df["ID"] = df["WS_src"]+"=>"+df["WS_dst"]
        df.drop(columns=["WS_src", "WS_dst"], inplace=True)

        df[df.columns[-1]]=df[df.columns[-1]].astype(str)
        IDS=sorted(list(df[df.columns[-1]].unique()))
        func_sw=[mean,std] #functions
        func_ew=[diff,
                 std_EW,sum_of_EW] #functions
        func_name_sw=["mean","std"]
        func_name_ew=["diff",
                    #   "mean_WE",
                      "std_WE",
                      "sum_of_EW"]
        f_list=[
            "pck_size",
            "ts",
            'TCP_window',
            ]
        fark=[
         'pck_size_mean',
         'pck_size_std',
         'ts_mean',
         'ts_std',
         'TCP_window_mean',
         'TCP_window_std',
         ]
        WS=windows_size
        flag=1
        for i in tqdm(IDS):
            # print('i',i)
            smaller = df[df["ID"]==i]
            smaller=smaller.reset_index()
            del smaller["index"]
            # ts = các ID trùng nhau trừ đi IP đầu,ID 1  - ID 0, ID 2 - ID 0
            smaller["ts"]=smaller["ts"].values-smaller["ts"].values[0]
            for ii in f_list: # đặt tên và gán giá trị ví dụ ts_mean , ts_std
                for jjj,iii in enumerate (func_ew):
                    name=str(ii)+"_"+str(func_name_ew[jjj])
                    smaller[name]=iii(smaller[ii])
            # có ID trùng nhau  >= WS mới tính sw
            if len(smaller)>=WS:
                # tính toán giá trị dùng roll window
                for ii in f_list:
                    for jjj,iii in enumerate (func_sw):
                        name=str(ii)+"_"+str(func_name_sw[jjj])
                        smaller[name]=iii(smaller[ii],WS)
            else:
                for ii in fark:
                        smaller[ii]=0
            smaller['TCP_FIN_sum'  ]=flagsum (smaller[ 'TCP_FIN'  ].values)
            smaller['TCP_SYN_sum'  ]=flagsum (smaller['TCP_SYN'  ].values)
            smaller['TCP_RST_sum'  ]=flagsum (smaller['TCP_RST'  ].values)
            smaller['TCP_PSH_sum'  ]=flagsum (smaller['TCP_PSH'  ].values)
            smaller['TCP_ACK_sum'  ]=flagsum (smaller['TCP_ACK'  ].values)
            smaller['TCP_URG_sum'  ]=flagsum (smaller['TCP_URG'  ].values)
            smaller['TCP_ECE_sum'  ]=flagsum (smaller['TCP_ECE'  ].values)
            smaller['TCP_CWR_sum'  ]=flagsum (smaller['TCP_CWR'  ].values)
            smaller["TCP_SYN_ratio"]=smaller["TCP_SYN"]/(smaller["TCP_SYN_sum"]+10e-20)
            col_list= ["TCP_FIN_sum","TCP_SYN_sum","TCP_RST_sum","TCP_PSH_sum","TCP_ACK_sum","TCP_URG_sum","TCP_ECE_sum","TCP_CWR_sum"]
            smaller['sum'] = smaller[col_list].sum(axis=1)
            smaller["TCP_ACK_SR"]=smaller["TCP_ACK"]/(smaller["sum"]+10e-20)
            smaller["TCP_ACK_R"]=smaller["TCP_ACK_sum"]/(smaller["sum"]+10e-20)

            if flag:
                smaller.to_csv(j[5:],header=True,index=False)
                flag=0
            else:
                smaller.to_csv(j[5:],header=False,index=False,mode="a")
        df=pd.read_csv(j[5:])
        features=df.columns
        count=[]
        bos=[]
        df=df.sort_values(by='Unnamed: 0')
        last_name=f'./{outputfolder}/last_{str(WS)}_{j[5:]}'
        df.to_csv(last_name,index=None)
        print(WS,"-------------------------------------------------------------------------------")


dos-synflooding-5-dec_FE.csv


100%|██████████| 10562/10562 [01:31<00:00, 115.50it/s]


2 -------------------------------------------------------------------------------
dos-synflooding-5-dec_FE.csv


100%|██████████| 10562/10562 [01:28<00:00, 118.69it/s]


6 -------------------------------------------------------------------------------
dos-synflooding-5-dec_FE.csv


100%|██████████| 10562/10562 [01:29<00:00, 117.37it/s]


9 -------------------------------------------------------------------------------


# MERGE SW FILES

In [70]:
files_add=find_the_way('./SW','FE.csv')
unique_file_names=[]
for i in files_add:
    bas=i.replace('_', '@', 1).find('_')+1
    if i[bas:] not in unique_file_names:
        unique_file_names.append(i[bas:])
for i in tqdm(unique_file_names):
    files_add=find_the_way('./SW',i)
    flag=1
    for ii in files_add:
        WS=ii.split("_")[1]
        print(WS)
        new_cols={'pck_size_mean':f'pck_size_mean_{WS}',
        'pck_size_std':f'pck_size_std_{WS}',
        'ts_mean':f'ts_mean_{WS}',
        'ts_std':f'ts_std_{WS}',
        'TCP_window_mean':f'TCP_window_mean_{WS}',
        'TCP_window_std':f'TCP_window_std_{WS}',
        # 'payload_bytes_mean':f'payload_bytes_mean_{WS}',
        # 'payload_bytes_std':f'payload_bytes_std_{WS}',
        # 'entropy_mean':f'entropy_mean_{WS}',
        # 'entropy_std':f'entropy_std_{WS}'
        }

        print(new_cols)
        if flag:
             df=pd.read_csv(ii)
             df.rename(columns = new_cols, inplace = True)
             flag=0
        else:
            adding=pd.read_csv(ii,usecols=list(new_cols.keys()))
            adding.rename(columns = new_cols, inplace = True)
            df = pd.concat([df, adding], axis=1)
        os.remove(ii)
    temp=i.replace("_FE.","_SW.")
    temp=f"./SW/{temp}"
    # y=df["Label"]
    # del df["Label"]
    # df["Label"]=y
    df.to_csv(temp,index=False)
    #print(list(df.columns))
    #print(f"{WS}\n\n\n\n")



<generator object walk at 0x000002C5BBA83570>


  0%|          | 0/1 [00:00<?, ?it/s]

<generator object walk at 0x000002C5BBA83570>
2
{'pck_size_mean': 'pck_size_mean_2', 'pck_size_std': 'pck_size_std_2', 'ts_mean': 'ts_mean_2', 'ts_std': 'ts_std_2', 'TCP_window_mean': 'TCP_window_mean_2', 'TCP_window_std': 'TCP_window_std_2'}
6
{'pck_size_mean': 'pck_size_mean_6', 'pck_size_std': 'pck_size_std_6', 'ts_mean': 'ts_mean_6', 'ts_std': 'ts_std_6', 'TCP_window_mean': 'TCP_window_mean_6', 'TCP_window_std': 'TCP_window_std_6'}
9
{'pck_size_mean': 'pck_size_mean_9', 'pck_size_std': 'pck_size_std_9', 'ts_mean': 'ts_mean_9', 'ts_std': 'ts_std_9', 'TCP_window_mean': 'TCP_window_mean_9', 'TCP_window_std': 'TCP_window_std_9'}


100%|██████████| 1/1 [00:00<00:00,  3.20it/s]
